<h1><center>LangChain Expression Language - LCEL</center></h1>
<hr><hr>
LCEL makes it easy to build complex chains from basic components, and supports out of the box functionality such as streaming, parallelism, and logging.

### Configurations:-
---------------------

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_api_version = "2023-05-15"
llm_deployment_name = os.getenv("GPT_DEPLOYMENT_NAME")

os.environ["OPENAI_API_TYPE"]     = "azure"
os.environ["OPENAI_API_VERSION"]  = azure_openai_api_version
os.environ["OPENAI_API_KEY"]      = azure_openai_api_key

## Basic example: prompt + model + output parser
---------------------------------------------------
The most basic and common use case is chaining a prompt template and a model together. To see how this works, let’s create a chain that takes a topic and generates a joke:

In [3]:
from langchain_openai import AzureChatOpenAI

chat_model = AzureChatOpenAI(
    openai_api_version=azure_openai_api_version,
    azure_deployment=llm_deployment_name,
)

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

joke_prompt = ChatPromptTemplate.from_template("""Tell me a joke about {joke_topic}""")
op_parser = StrOutputParser()

chain = joke_prompt | chat_model | op_parser

In [ ]:
response = chain.invoke(  )